In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps

# Project: Exploring the Hubble "Tension"

Recall that in weeks 1 and 2 of this course we explored some measurements of the Hubble constant. We used them to study some simple statistics and make an inverse variance weighted average of measurements that took into account the fact that some of the measurements had smaller uncertainties than others.

It turns out that the inconsistency in the measurements of the Hubble constant is actually one of the biggest open questions in cosmology.  Roughly speaking, measurements of the Hubble constant can be divided into two types. 

1. "Early time" measurements that use our knowledge of cosmology and look at structures in the very early universe, e.g., the primoridal background of light left over from the Big Bang that is now visible to us as a bath of faint microwave radiation, to determine the Hubble parameter.  These measurements are more sensitive to our understanding of cosmology.

2. "Late time" measurements that build on the so-called distance ladder to estimate the distance to supernovae and compare that to the redshift of their spectral lines (i.e., the sort of measurement we did in week 6 looking at the spectrum of a Galaxy using SDSS data).  These measurements are more sensitive to our ability to build up a distance ladder out to billions of light years away.

We've provided you with a new data file, which is a sub-set of the more recent Hubble data measurements, including an extra column telling you if a particular measurement is type 1 or type 2.

We expect you to complete all of the following goals for full credit. If you prefer to replace on of these goals with your own goal, please consult the instructor/TA.

1. Come up with a way to evaluate the consistency of all of the data before subdividing into "Early Time" and "Late Time" measurements. Make sure you take the errors into account, quote a p-value (probability estimate), and explain why your results indicate the data should be subdivided. 

2. Make a nice plot (or two) that summarizes the difference between "Early Time" results and "Late Time" results.

3. Come up with a way to evaluate the consistency of the data within each subset ("Early Time" and "Late Time"). Make sure you take the errors into account, quote a p-value (probability estimate), and interpret your result.

4. Come up with a way to evaluate the consistency of the two different subsets ("Early Time" and "Late Time") with each other. Make sure you take the errors into account, quote a p-value (probability estimate), and interpret your result.

In [ ]:
data = np.loadtxt(open("../data/Hubble_extra.txt", 'rb'), usecols=[1,2,3,4])

In [ ]:
# This is how we pull out the data from columns in the array.
H0_measured = data[:,0]
H0_errorLow = data[:,1]
H0_errorHigh = data[:,2]
H0_type = data[:,3]
N_measurements = H0_measured.size

# Useful functions

The first one returns useful statistical quantities, given arrays of values and associated error estimates.

The second one takes a set of Hubble parameter measurements and their associated error bars (both low and high side) and uses the first function to print out the statistical quantities, then makes a plot of the measurements.

In [ ]:
def getStats(values, errors):
    nvals = np.size(values)
    mean = np.mean(values)
    std = np.std(values)
    err = std / np.sqrt(nvals)
    wts = 1/(errors*errors)
    wmean = np.sum(values*wts)/np.sum(wts)
    werr = np.sqrt(1/np.sum(wts))
    return (nvals, mean, std, err, wmean, werr)

In [ ]:
def getStatsAndPlotValues(values, errs_lo, errs_hi):
    errs = 0.5*(errs_lo + errs_hi)
    stats = getStats(values, errs)
    print("Statistics for these data ----")
    print("N                      : %i" % stats[0])
    print("Mean                   : %0.2f" % stats[1])
    print("Standard Deviation     : %0.2f" % stats[2])
    print("Standard Error         : %0.2f" % stats[3])
    print("Weighted Mean          : %0.2f" % stats[4])
    print("Error on Weighted Mean : %0.2f" % stats[5])
    _ = plt.errorbar(values, np.arange(stats[0]), xerr=(errs_lo, errs_hi), fmt=".", color='k')
    _ = plt.xlabel("Hubble Constant [km/s/Mpc]")
    _ = plt.ylabel("Experiment number")
    _ = plt.errorbar(stats[1], stats[0]/2., xerr=stats[3], yerr=stats[0]/2, fmt='o', color='b', label="Mean")
    _ = plt.errorbar(stats[4], stats[0]/2., xerr=stats[5], yerr=stats[0]/2, fmt='o', color="r", label="Weighted Mean")
    _ = plt.legend()

In [ ]:
getStatsAndPlotValues(H0_measured, H0_errorLow, H0_errorHigh)

# Masking data by measurement type

This next cell shows how to select measurements of either type

In [ ]:
type_1_mask = H0_type == 1
type_2_mask = H0_type == 2
print("Early time measurements", H0_measured[type_1_mask])
print("Late time measurements", H0_measured[type_2_mask])

# Going from significance in sigma to p-value and vice versa

These functions might be useful.

In [ ]:
sigma_value = 3
p_value = sps.norm().sf(sigma_value)
sigma_check = sps.norm().isf(p_value)
print("Original sigma : %.2f" % sigma_value)
print("P-value        : %.2e" % p_value)
print("Back to sigma  : %.2f" % sigma_check)

# Going from a chi-squared to a p-value

This function tells you the p-value of observing a given total chi squared value given df = n-1 measurements.

Where df is the "degrees of freedom," so if we have, say, 11 measurements but are picking one value for the Hubble constant to try and match all the data, that leaves us with 10 "degrees of freedom".

I.e., if we had 11 measurements, and the total chi squared summed up to 11.5, then the p-value would be about 0.32

In [ ]:
print("The p-value for chi squared = 11.5 with 10 degrees of freedom is %0.2f" % sps.chi2(df=10).sf(11.5))